# Checking Image Histogram

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import seaborn as sns
import os

In [ ]:
folder = './test_images'
test_images = [fn_img for fn_img in os.listdir(folder) if fn_img.startswith('test_img_')]

In [ ]:
hist_rgb_sum = np.zeros((256,256,256))
hist_hsv_sum = np.zeros((256,256,256))

for fn_test in test_images:
    fn_test_full = '{0}/{1}'.format(folder, fn_test)
    fn_mask_full = '{0}/mask_{1}'.format(folder, fn_test)
    img = cv2.cvtColor(cv2.imread(fn_test_full),cv2.COLOR_BGR2RGB)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    img_mask = cv2.cvtColor(cv2.imread(fn_mask_full), cv2.COLOR_BGR2GRAY)
    hist_rgb = cv2.calcHist([img],
            [0,1,2],
            img_mask, 
            [256] * 3,
            [0,256] * 3)
    hist_hsv = cv2.calcHist([img_hsv],
            [0,1,2],
            img_mask, 
            [256] * 3,
            [0,256] * 3)
    hist_rgb_sum += hist_rgb
    hist_hsv_sum += hist_hsv

# RGB Histogram

In [ ]:
rgb_sum_nonzero = np.nonzero(hist_rgb_sum)

list_rgb = []

for h, s, v in zip(rgb_sum_nonzero[0], rgb_sum_nonzero[1], rgb_sum_nonzero[2]):
    list_rgb.append([h,s,v,hist_rgb_sum[h,s,v]])

np_list_hist_rgb = np.array(list_rgb)

In [ ]:
df_rgb_sum = pd.DataFrame(np_list_hist_rgb,
                          columns=['R','G','B','cnt']).astype(np.uint8)

In [ ]:
fig,axes=plt.subplots(1,3, figsize=(18,10))
cbar_set = dict(orientation="horizontal")
sns.histplot(ax=axes[0],
             data=df_rgb_sum, 
             x="R", y="G", 
             bins=50, pthresh=.1, cmap="viridis",cbar=True,
             cbar_kws = cbar_set)
sns.histplot(ax=axes[1],
             data=df_rgb_sum, 
             x="G", y="B", 
             bins=50, pthresh=.1, cmap="viridis",cbar=True,
             cbar_kws = cbar_set)
sns.histplot(ax=axes[2],
             data=df_rgb_sum, 
             x="R", y="B", 
             bins=50, pthresh=.1, cmap="viridis",cbar=True,
             cbar_kws = cbar_set)
fig.tight_layout()

# Create Threshold Filter with Given Histogram

In [ ]:
arr_RG = axes[0].get_children()[0].get_array()
np_arr_RG = np.reshape(arr_RG.data,(50,50))

arr_GB = axes[1].get_children()[0].get_array()
np_arr_GB = np.reshape(arr_GB.data,(50,50))

arr_RB = axes[1].get_children()[0].get_array()
np_arr_BR = np.reshape(arr_RB.data,(50,50))

In [ ]:
plt.imshow(np_arr_RG)

In [ ]:
list_arr_RG = []
thres = 5
for i in range(0,50):
    nonzeros = np.where(np_arr_RG[:,i] > thres)[0]
    if(len(nonzeros)>0):
        try:
            minval = (np.min(nonzeros) + 1) * 5
            maxval = (np.max(nonzeros) + 1) * 5 + 5
        except:
            print(nonzeros)
    else:
        minval = -1
        maxval = -1
    list_arr_RG.append([i*5, (i+1)*5, minval, maxval])

In [ ]:
list_arr_RG

# HSV Histogram

In [ ]:
hsv_sum_nonzero = np.nonzero(hist_hsv_sum)

list_hsv = []

for h, s, v in zip(hsv_sum_nonzero[0], hsv_sum_nonzero[1], hsv_sum_nonzero[2]):
    list_hsv.append([h,s,v,hist_hsv_sum[h,s,v]])

np_list_hist_hsv = np.array(list_hsv)

In [ ]:
df_hsv_sum = pd.DataFrame(np_list_hist_hsv,
                          columns=['H','S','V','cnt']).astype(np.uint8)

In [ ]:
fig,axes=plt.subplots(1,3, figsize=(18,10))
cbar_set = dict(orientation="horizontal")
sns.histplot(ax=axes[0],
             data=df_hsv_sum, 
             x="H", y="S", 
             bins=50, pthresh=.1, cmap="viridis",cbar=True,
             cbar_kws = cbar_set)
sns.histplot(ax=axes[1],
             data=df_hsv_sum, 
             x="S", y="V", 
             bins=50, pthresh=.1, cmap="viridis",cbar=True,
             cbar_kws = cbar_set)
sns.histplot(ax=axes[2],
             data=df_hsv_sum, 
             x="H", y="V", 
             bins=50, pthresh=.1, cmap="viridis",cbar=True,
             cbar_kws = cbar_set)
fig.tight_layout()